In [64]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

>Vamos a realizar por cada funcion una exportacion de los datos que la funcion debe consumir, ya listos, en un nuevo archivo.parquet. Luego crearemos la funcion de cosulta necesaria para recuperar los datos.

**def Developer( desarrollador: str )**

Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

**Ejemplo de uso:**

```json
{
 'Año': [2008],
 'Cantidad de Items': [1],
 'Contenido Free': [100.0]
}


Exportacion de datos

In [65]:
# Ruta del archivo Parquet
archivo_parquet = 'datasets/processed_data/steam_games.parquet'

# Utiliza PyArrow para leer el archivo Parquet
tabla = pq.read_table(archivo_parquet)
df_steam_games = tabla.to_pandas()

# Agrupar por desarrollador y año
grouped_data = df_steam_games.groupby(["developer", "release_year"])

# Calcular la cantidad total de ítems y la cantidad de ítems gratuitos para cada desarrollador y año
total_items = grouped_data["item_id"].nunique()
free_items = grouped_data.apply(lambda x: (x["price"] == 0.00).sum())

# Calcular el porcentaje de ítems gratuitos
free_percentage = (free_items / total_items * 100).round(2).fillna(0)

# Crear el DataFrame final
result_df = pd.DataFrame({
    "Desarrollador": total_items.index.get_level_values(0),
    "Año": total_items.index.get_level_values(1),
    "Cantidad de Items": total_items.values,
    "Contenido Free": free_percentage.values
})

# Guardar el DataFrame en un archivo parquet
result_df.to_parquet('Developer.parquet', index=False)

C:\Users\sebas\AppData\Local\Temp\ipykernel_692\2189582388.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  free_items = grouped_data.apply(lambda x: (x["price"] == 0.00).sum())


Creamos la funcion

In [66]:
def Developer(desarrollador: str):
    # Cargar los datos desde el archivo Parquet
    file_path = "datasets/processed_data/data_for_funciones/Developer.parquet"
    table = pq.read_table(file_path)
    df = table.to_pandas()
    
    # Filtrar el DataFrame por el desarrollador especificado
    filtered_df = df[df['Desarrollador'] == desarrollador]
    
    # Calcular la cantidad total de items por año y el porcentaje de contenido gratuito
    result = filtered_df.groupby('Año').agg({'Cantidad de Items': 'sum', 'Contenido Free': 'mean'}).reset_index()
    
    # Convertir los resultados a un diccionario en el formato deseado
    output = {
        'Año': result['Año'].tolist(),
        'Cantidad de Items': result['Cantidad de Items'].tolist(),
        'Contenido Free': result['Contenido Free'].tolist()
    }
    
    return output



In [67]:
# Ejemplo de uso
Developer("Valve")

{'Año': [1998,
  1999,
  2000,
  2001,
  2003,
  2004,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2017],
 'Cantidad de Items': [1, 1, 2, 1, 1, 5, 2, 3, 1, 1, 2, 1, 1, 2],
 'Contenido Free': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  33.33,
  0.0,
  0.0,
  50.0,
  0.0,
  0.0,
  0.0]}

**def UserData( user_id: str )**

Esta funcion toma como argumento un user_id y devuelve cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

**Ejemplo de retorno:**

```json
{
  "Usuario X": "us213ndjss09sdf",
  "Dinero gastado": "200 USD",
  "porcentaje de recomendación": "20%",
  "Cantidad de items": 5
}

Exportacion de datos

In [70]:

# Ruta de los archivos Parquet
games_parquet = 'datasets/processed_data/steam_games.parquet'
reviews_parquet = 'datasets/processed_data/user_reviews.parquet'

# Utiliza PyArrow para leer el archivo Parquet
tabla_games = pq.read_table(games_parquet)
tabla_reviews = pq.read_table(reviews_parquet)

# Leer las tablas en DataFrames de Pandas
df_steam_games = tabla_games.to_pandas()
df_user_reviews = tabla_reviews.to_pandas()

# Merge de los DataFrames utilizando "item_id" como clave
merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

# Lista de user_id únicos
unique_user_ids = df_user_reviews['user_id'].unique()

# Lista para almacenar todos los DataFrames de resultados
all_results_dfs = []

# Iterar sobre cada user_id y obtener la información
for user_id in unique_user_ids:
    user_data = merged_df[merged_df['user_id'] == user_id]

    # Calcular dinero gastado
    total_spent = round(user_data['price'].sum(), 2)

    # Calcular el porcentaje de recomendación
    total_items = user_data['item_id'].nunique()
    if total_items != 0:
        recommend_percentage = (user_data['recommend'].sum() / total_items) * 100
    else:
        recommend_percentage = 0

    # Crear un DataFrame con los resultados
    result_df = pd.DataFrame({
        "Usuario X": [user_id],
        "Dinero gastado": [f"{total_spent} USD"],
        "porcentaje de recomendación": [f"{recommend_percentage:.2f}%"],
        "Cantidad de items": [total_items]
    })

    # Agregar el DataFrame a la lista
    all_results_dfs.append(result_df)

# Concatenar todos los DataFrames en uno solo
all_results_df = pd.concat(all_results_dfs, ignore_index=True)

In [73]:
# Guardar el DataFrame en un archivo parquet
all_results_df.to_parquet('UserData.parquet', index=False)

Creamos funcion

In [76]:
def UserData(user_id: str):
    file_path = 'datasets/processed_data/data_for_funciones/UserData.parquet'
    table = pq.read_table(file_path)
    user_data = table.to_pandas()
    
    user_info = user_data[user_data['Usuario X'] == user_id].to_dict(orient='records')
    
    if user_info:
        return user_info[0]
    else:
        return f"No se encontraron datos para el usuario con ID: {user_id}"

In [77]:
UserData("evcentric")


{'Usuario X': 'evcentric',
 'Dinero gastado': '68.94 USD',
 'porcentaje de recomendación': '100.00%',
 'Cantidad de items': 6}

**def UserForGenre( genero: str )**

Devuelve el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

**Ejemplo de retorno:**

```json
{
  "Usuario con más horas jugadas para Género X": "us213ndjss09sdf",
  "Horas jugadas": [
    {"Año": 2013, "Horas": 203},
    {"Año": 2012, "Horas": 100},
    {"Año": 2011, "Horas": 23}
  ]
}


Exportamos datos

In [86]:

# Ruta de los archivos Parquet
games_parquet = "datasets/processed_data/steam_games.parquet"
items_parquet = "datasets/processed_data/user_items.parquet"

# Utiliza PyArrow para leer el archivo Parquet
tabla_games = pq.read_table(games_parquet)
tabla_items = pq.read_table(items_parquet)

# Leer las tablas en DataFrames de Pandas
df_steam_games = tabla_games.to_pandas()
df_user_items = tabla_items.to_pandas()

# Lista de géneros a analizar
generos = ['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling',
    'Audio Production', 'Casual', 'Design &amp; Illustration',
    'Early Access', 'Education', 'Free to Play', 'Indie',
    'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation',
    'Software Training', 'Sports', 'Strategy', 'Utilities',
    'Video Production', 'Web Publishing']

# Lista para almacenar los resultados de cada género
resultados = []

# Iterar sobre cada género
for genero in generos:
    # Filtra los juegos por el género especificado
    juegos_filtrados = df_steam_games[df_steam_games['genres'].str.contains(genero, case=False, na=False)]

    # Fusiona df_user_items y df_games basándose en item_id e id
    df_fusionado = pd.merge(df_user_items, juegos_filtrados, left_on='item_id', right_on='item_id', how='inner')

    # Agrupa por año y usuario, calcula el tiempo de juego por usuario por año
    agrupado = df_fusionado.groupby(['release_year', 'user_id'])['playtime_forever'].sum().reset_index()

    if len(agrupado) > 0:
        # Encuentra al usuario con más tiempo de juego para el género dado
        usuario_max = agrupado[agrupado['playtime_forever'] == agrupado.groupby('release_year')['playtime_forever'].transform('max')]['user_id'].values[0]

        # Filtra los datos para el usuario con más tiempo de juego
        datos_usuario = agrupado[agrupado['user_id'] == usuario_max]

        # Elimina los años con 0 tiempo de juego
        datos_usuario = datos_usuario[datos_usuario['playtime_forever'] > 0]

        # Ordena los años en orden descendente
        datos_usuario = datos_usuario.sort_values(by='release_year', ascending=False)

        # Convierte el tiempo de juego a enteros
        datos_usuario['playtime_forever'] = datos_usuario['playtime_forever'].astype(int)

        # Crea una lista de tiempo de juego acumulado por año
        horas_por_año = [{'Año': int(año), 'Horas': int(horas)} for año, horas in zip(datos_usuario['release_year'], datos_usuario['playtime_forever'])]

        # Añadir el resultado a la lista de resultados
        resultados.append({"genero": genero, "usuario con mas horas jugadas": usuario_max, "Horas jugadas": horas_por_año})
    else:
        resultados.append({"genero": genero, "usuario con mas horas jugadas": None, "Horas jugadas": []})

# Convertir la lista de resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)


In [94]:
# Eliminar filas con valores nulos en el DataFrame
df_resultados = df_resultados.dropna()

genero                           0
usuario con mas horas jugadas    0
Horas jugadas                    0
dtype: int64

In [95]:
# Guardar el DataFrame en un archivo parquet
df_resultados.to_parquet('UserForGenre.parquet', index=False)

Creamos funcion

In [106]:
def UserForGenre(genero: str):
    # Cargar los datos desde el archivo Parquet
    table = pq.read_table('datasets/processed_data/data_for_funciones/UserForGenre.parquet')
    data = table.to_pandas()

    # Filtrar los datos por género
    genre_data = data[data['genero'] == genero]

    # Obtener el usuario con más horas jugadas para el género dado
    max_hours_user = genre_data.loc[genre_data['Horas jugadas'].idxmax(), 'usuario con mas horas jugadas']

    # Obtener las horas jugadas para el género dado
    horas_jugadas = genre_data['Horas jugadas'].tolist()

    # Formatear el resultado
    result = {
        "Usuario con más horas jugadas para {}".format(genero): max_hours_user,
        "Horas jugadas": horas_jugadas
    }

    return result



In [107]:
# Ejemplo de uso
UserForGenre("Action")

{'Usuario con más horas jugadas para Action': '76561197966936422',
 'Horas jugadas': [array([{'Año': 2016, 'Horas': 1366}, {'Año': 2015, 'Horas': 9708},
         {'Año': 2014, 'Horas': 1416}, {'Año': 2013, 'Horas': 2095},
         {'Año': 2012, 'Horas': 6380}, {'Año': 2011, 'Horas': 129},
         {'Año': 2010, 'Horas': 36}, {'Año': 2009, 'Horas': 69},
         {'Año': 2008, 'Horas': 13}, {'Año': 2007, 'Horas': 139},
         {'Año': 2006, 'Horas': 15}, {'Año': 2004, 'Horas': 209},
         {'Año': 2000, 'Horas': 10}, {'Año': 1998, 'Horas': 3},
         {'Año': 1983, 'Horas': 331}], dtype=object)]}

**def Best_Developer_Year( año: int )**

Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

**Ejemplo de retorno:**

```json
[
  {"Puesto 1": "desarrollador_X"},
  {"Puesto 2": "desarrollador_Y"},
  {"Puesto 3": "desarrollador_Z"}
]


Exportacion

In [108]:
# Ruta de los archivos Parquet
games_parquet = "datasets/processed_data/steam_games.parquet"
reviews_parquet = "datasets/processed_data/user_reviews.parquet"


In [112]:
#Listamos los años unicos de nuestros datos
unique_release_years = df_steam_games['release_year'].unique()
print(unique_release_years)


[2018 2017 1997 1998 2016 2006 2005 2003 2007 2002 2000 1995 1996 1994
 2001 1993 2004 2008 2009 1999 1992 1989 2010 2011 2013 2012 2014 1983
 1984 2015 1990 1988 1991 1987 1986 2021 2019 1985]


In [146]:

# Utiliza PyArrow para leer el archivo Parquet
tabla_games = pq.read_table(games_parquet)
tabla_reviews = pq.read_table(reviews_parquet)

# Leer las tablas en DataFrames de Pandas
df_steam_games = tabla_games.to_pandas()
df_user_reviews = tabla_reviews.to_pandas()

anios = [2018, 2017, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002, 2000, 1995, 1996, 1994,
2001, 1993, 2004, 2008, 2009, 1999, 1992, 1989, 2010, 2011, 2013, 2012, 2014, 1983,
1984, 2015, 1990, 1988, 1991, 1987, 1986, 2021, 2019, 1985]

resultados = []

for anio in anios:
    # Merge de los DataFrames utilizando "item_id" como clave
    merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

    # Filtrar el DataFrame por el anio dado
    df_filtered = merged_df[merged_df['release_year'] == anio]

    # Filtrar por juegos recomendados y comentarios positivos
    df_filtered = df_filtered[(df_filtered['recommend'] == True) & (df_filtered['sentiment_analysis'] == 2.0)]

    # Agrupar por desarrollador y contar la cantidad de juegos recomendados
    developer_counts = df_filtered.groupby('developer')['item_id'].count()

    # Obtener los tres mejores desarrolladores
    top_developers = developer_counts.nlargest(3)

    # Crear el formato de salida
    output = {"Anio": anio, "Desarrolladores": [{"Puesto {}: ".format(i+1) + developer: count} for i, (developer, count) in enumerate(top_developers.items())]}
    
    resultados.append(output)
    
df_resultados = pd.DataFrame(resultados)

In [161]:
# Filtrar filas con listas vacías en la columna 'Desarrolladores'
df_resultados = df_resultados[df_resultados['Desarrolladores'].apply(lambda x: len(x) > 0)]

In [162]:
df_resultados

,Anio,Desarrolladores
1,2017,"[{'Puesto 1: Smartly Dressed Games': 827}, {'P..."
2,1997,"[{'Puesto 1: Oddworld Inhabitants': 17}, {'Pue..."
3,1998,"[{'Puesto 1: Valve': 50}, {'Puesto 2: Black Is..."
4,2016,"[{'Puesto 1: Chucklefish': 304}, {'Puesto 2: R..."
5,2006,"[{'Puesto 1: Facepunch Studios': 1499}, {'Pues..."
6,2005,"[{'Puesto 1: Pandemic Studios': 82}, {'Puesto ..."
7,2003,"[{'Puesto 1: Running With Scissors': 61}, {'Pu..."
8,2007,"[{'Puesto 1: Valve': 3408}, {'Puesto 2: 2K Bos..."
9,2002,"[{'Puesto 1: Bethesda Game Studios': 13}, {'Pu..."
10,2000,"[{'Puesto 1: Valve': 60}, {'Puesto 2: Ion Stor..."


In [163]:
# Guardar el DataFrame en un archivo parquet
df_resultados.to_parquet('Best_Developer_Year.parquet', index=False)

crear funcion

In [179]:
import ast

def Best_Developer_Year(año: int):
    # Ruta de los archivos Parquet
    Best_Developer_Year = 'datasets/processed_data/data_for_funciones/Best_Developer_Year.Parquet'

    # Utiliza PyArrow para leer el archivo Parquet
    tabla_Best_Developer_Year = pd.read_parquet(Best_Developer_Year)

    # Filtrar por año
    data_año = tabla_Best_Developer_Year[tabla_Best_Developer_Year['Anio'] == año]

    # Obtener los desarrolladores para el año dado
    if not data_año.empty:
        desarrolladores = data_año['Desarrolladores'].iloc[0]
        # Convertir la matriz numpy a lista de diccionarios
        desarrolladores = desarrolladores.tolist()
        desarrolladores = ast.literal_eval(str(desarrolladores))
        return [{"Puesto " + str(i+1): list(d.keys())[0]} for i, d in enumerate(desarrolladores)]
    else:
        return "No se encontraron datos para el año especificado."


In [177]:
Best_Developer_Year(2010)

[{'Puesto 1': 'Puesto 1: 3D Realms (Apogee Software)'},
 {'Puesto 2': 'Puesto 1: 3D Realms (Apogee Software)'},
 {'Puesto 3': 'Puesto 1: 3D Realms (Apogee Software)'}]

**def Developer_Reviews_Analysis( desarrolladora: str )**

Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

**Ejemplo de retorno:**

```json
{
  "Valve": {
    "Negative": 182,
    "Positive": 278
  }
}


exportacion

In [126]:
# Ruta de los archivos Parquet
games_parquet = "datasets/processed_data/steam_games.parquet"
reviews_parquet = "datasets/processed_data/user_reviews.parquet"

# Utiliza PyArrow para leer el archivo Parquet
tabla_games = pq.read_table(games_parquet)
tabla_reviews = pq.read_table(reviews_parquet)

# Leer las tablas en DataFrames de Pandas
df_steam_games = tabla_games.to_pandas()
df_user_reviews = tabla_reviews.to_pandas()

# Merge de los DataFrames utilizando "item_id" como clave
merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

# Agrupar por desarrolladora y contar las reseñas positivas y negativas
df_grouped = merged_df.groupby('developer')['sentiment_analysis'].value_counts().unstack(fill_value=0)

# Reiniciar el índice para convertir la columna 'developer' en una columna regular
df_grouped.reset_index(inplace=True)

# Renombrar las columnas
df_grouped.columns.name = None  # Eliminar el nombre de las columnas para una mejor presentación
df_grouped.rename(columns={1.0: 'Negative', 2.0: 'Positive'}, inplace=True)



In [129]:
# Guardar el DataFrame en un archivo parquet
df_grouped.to_parquet('Developer_Reviews_Analysi.parquet', index=False)
df_grouped.to_csv('Developer_Reviews_Analysi.csv', index=False)

crear funcion

In [130]:
import pandas as pd

def Developer_Reviews_Analysis(desarrolladora: str):
    table = pq.read_table('datasets/processed_data/data_for_funciones/Developer_Reviews.parquet')
    df = table.to_pandas()
    
    # Filtrar el DataFrame para obtener las filas correspondientes a la desarrolladora proporcionada
    filtered_df = df[df['developer'] == desarrolladora]
    
    # Calcular la suma de las reseñas positivas y negativas
    total_negative = filtered_df['Negative'].sum()
    total_positive = filtered_df['Positive'].sum()
    
    # Crear el diccionario de retorno
    result = {
        desarrolladora: {
            "Negative": total_negative,
            "Positive": total_positive
        }
    }
    
    return result

# Ejemplo de uso
print(Developer_Reviews_Analysis("Valve"))


{'Valve': {'Negative': 10, 'Positive': 8353}}
